In [1]:
# Importing all the necessary libraries

import pandas as pd
import numpy as np
import nltk
from wordcloud import WordCloud
from wordcloud import ImageColorGenerator
from wordcloud import STOPWORDS
import matplotlib.pyplot as plt

In [7]:
df_1 = pd.read_csv(r'C:\Users\Nemo\Downloads\Jules Verne\df_twenty_thousand_labelled.csv')
df_2 = pd.read_csv(r'C:\Users\Nemo\Downloads\Jules Verne\df_journey_to_center_labelled.csv')
df_3 = pd.read_csv(r'C:\Users\Nemo\Downloads\Jules Verne\df_mysterious_island_labelled.csv')
df_4 = pd.read_csv(r'C:\Users\Nemo\Downloads\Jules Verne\df_round_the_moon_labelled.csv')
df_5 = pd.read_csv(r'C:\Users\Nemo\Downloads\Jules Verne\df_from_earth_to_moon_labelled.csv')
df_test = pd.read_csv(r'C:\Users\Nemo\Downloads\Jules Verne\df_around_the_world_labelled.csv')

In [8]:
df_train = pd.concat([df_1, df_2, df_3, df_4, df_5], axis=0)

In [9]:
df_train['StopwordRemovedSentence'].isnull().sum()

158

In [10]:
df_test['StopwordRemovedSentence'].isnull().sum()

14

In [11]:
df_train = df_train[pd.notnull(df_train['StopwordRemovedSentence'])]
df_train = df_train.reset_index(drop=True)

df_test = df_test[pd.notnull(df_test['StopwordRemovedSentence'])]
df_test = df_test.reset_index(drop=True)

In [12]:
df_train['StopwordRemovedSentence'].isnull().sum()

0

In [13]:
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer

train_og_review = df_train['StopwordRemovedSentence']
test_og_review = df_test['StopwordRemovedSentence']

cv = CountVectorizer(ngram_range = [1,1]) #for unigrams

In [14]:
train_vector = cv.fit_transform(train_og_review)
print("Train Vector Shape:",train_vector.shape)

test_vector = cv.transform(test_og_review)
print("Test Vector Shape:", test_vector.shape)

Train Vector Shape: (23089, 17068)
Test Vector Shape: (2753, 17068)


In [15]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix

# define true labels from train set
x_train = train_vector
y_train = df_train["Label"]
x_test = test_vector
y_test = df_test["Label"]

# fit model and test on trai


model = MultinomialNB()
model.fit(x_train, y_train)
predictions = model.predict(x_test)
print ("Accuracy score: ", accuracy_score(y_test, predictions))

Accuracy score:  0.7134035597529967


In [16]:
print ("Precision score: ", precision_score(y_test, predictions, average = 'weighted'))
print ("Recall score: ", recall_score(y_test, predictions, average = 'weighted'))
print ("F1 score: ", f1_score(y_test, predictions, average = 'weighted'))

Precision score:  0.7178708065297873
Recall score:  0.7134035597529967
F1 score:  0.711875228116898


#### Task 2 (feature selection 1): Remove features with low variance

In [17]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold = 0.001)

X_train_features_filtered_var_thr = selector.fit(train_vector).transform(train_vector)
print ("Train feature space before filtering: ", train_vector.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_var_thr.shape)


X_test_features_filtered_var_thr = selector.transform(test_vector)
print ("Test feature space before filtering: ", test_vector.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_var_thr.shape)

Train feature space before filtering:  (23089, 17068)
Train feature space after filtering:  (23089, 1878)
Test feature space before filtering:  (2753, 17068)
Test feature space after filtering:  (2753, 1878)


In [19]:
# define true labels from train set
x_train = X_train_features_filtered_var_thr
y_train = df_train["Label"]
x_test = X_test_features_filtered_var_thr
y_test = df_test["Label"]



# fit model and test on train
model = MultinomialNB()
model.fit(x_train, y_train)
predictions = model.predict(x_test)
print ("Accuracy score: ", accuracy_score(y_test, predictions))

Accuracy score:  0.6749001089720306


In [20]:
# For average = WEIGHTED
print ("Precision score: ", precision_score(y_test, predictions, average = 'weighted'))
print ("Recall score: ", recall_score(y_test, predictions, average = 'weighted'))
print ("F1 score: ", f1_score(y_test, predictions, average = 'weighted'))

Precision score:  0.6781649294373491
Recall score:  0.6749001089720306
F1 score:  0.670623772290847


In [21]:
selector = VarianceThreshold(threshold = 0.005)

X_train_features_filtered_var_thr = selector.fit(train_vector).transform(train_vector)
print ("Train feature space before filtering: ", train_vector.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_var_thr.shape)


X_test_features_filtered_var_thr = selector.transform(test_vector)
print ("Test feature space before filtering: ", test_vector.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_var_thr.shape)

Train feature space before filtering:  (23089, 17068)
Train feature space after filtering:  (23089, 384)
Test feature space before filtering:  (2753, 17068)
Test feature space after filtering:  (2753, 384)


In [22]:
# define true labels from train set
x_train = X_train_features_filtered_var_thr
y_train = df_train["Label"]
x_test = X_test_features_filtered_var_thr
y_test = df_test["Label"]



# fit model and test on train
model = MultinomialNB()
model.fit(x_train, y_train)
predictions = model.predict(x_test)
print ("Accuracy score: ", accuracy_score(y_test, predictions))

Accuracy score:  0.5608427170359608


In [23]:
# For average = WEIGHTED
print ("Precision score: ", precision_score(y_test, predictions, average = 'weighted'))
print ("Recall score: ", recall_score(y_test, predictions, average = 'weighted'))
print ("F1 score: ", f1_score(y_test, predictions, average = 'weighted'))

Precision score:  0.5855488858976261
Recall score:  0.5608427170359608
F1 score:  0.538234452051724


#### Task 3 (feature selection 2): Select top k-best features using chi-squared statistics

In [24]:
from sklearn.feature_selection import SelectKBest, chi2

selector = SelectKBest(chi2, k='all')
X_train_features_filtered_kbest = selector.fit_transform(train_vector, y_train)
print ("Train feature space before filtering: ", train_vector.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_kbest.shape)


X_test_features_filtered_kbest = selector.transform(test_vector)
print ("Test feature space before filtering: ", test_vector.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_kbest.shape)

Train feature space before filtering:  (23089, 17068)
Train feature space after filtering:  (23089, 17068)
Test feature space before filtering:  (2753, 17068)
Test feature space after filtering:  (2753, 17068)


In [25]:
all_train_features = cv

indexes = selector.get_support(indices=True)
scores = selector.scores_
feature_names = all_train_features#selector.get_feature_names_out()
print(len(scores), train_vector.shape[1])
print("\n")
print ("Total number of features selected: ", len(indexes))


#Finding informativeness of all the features
informative_features = []
for index in indexes:
    informative_features.append((cv.get_feature_names()[index],scores[index])) 

# Printing all of the features with increment of 40    
print("\n")
print("Informative features: ",informative_features[0::400])

17068 17068


Total number of features selected:  17068


Informative features:  [('000', 0.9997319519743658), ('accidents', 5.658369566905264), ('amethysts', 1.4969179193251867), ('asks', 3.177492310475846), ('bearded', 1.7767889356584488), ('bounded', 1.116004070971444), ('capes', 1.2901597436535512), ('chop', 1.477140623067147), ('compensate', 3.5535778713168975), ('coral', 1.9526026875384488), ('danes', 1.4969179193251867), ('deviation', 0.3608806093291058), ('dragonnier', 1.7767889356584488), ('encountered', 3.177492310475846), ('exotic', 1.7767889356584488), ('fight', 41.03513713465864), ('francisco', 2.1779194315081534), ('gnawed', 1.4959190242217544), ('hatch', 1.7767889356584488), ('humour', 3.177492310475846), ('inevitable', 2.3131832295378127), ('israelites', 5.330366806975345), ('laying', 5.853491912119066), ('macciota', 1.7767889356584488), ('mercy', 157.1112543138923), ('mushrooms', 2.8720045850628058), ('oblivion', 3.177492310475846), ('palisades', 3.3402139325658844), (

In [26]:
# define true labels from train set
x_train = X_train_features_filtered_var_thr
y_train = df_train["Label"]
x_test = X_test_features_filtered_var_thr
y_test = df_test["Label"]



# fit model and test on train
model = MultinomialNB()
model.fit(x_train, y_train)
predictions = model.predict(x_test)
print ("Accuracy score: ", accuracy_score(y_test, predictions))

Accuracy score:  0.5608427170359608


In [27]:
# For average = WEIGHTED
print ("Precision score: ", precision_score(y_test, predictions, average = 'weighted'))
print ("Recall score: ", recall_score(y_test, predictions, average = 'weighted'))
print ("F1 score: ", f1_score(y_test, predictions, average = 'weighted'))

Precision score:  0.5855488858976261
Recall score:  0.5608427170359608
F1 score:  0.538234452051724


In [28]:
selector = SelectKBest(chi2, k=1000)
X_train_features_filtered_kbest = selector.fit_transform(train_vector, y_train)
print ("Train feature space before filtering: ", train_vector.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_kbest.shape)


X_test_features_filtered_kbest = selector.transform(test_vector)
print ("Test feature space before filtering: ", test_vector.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_kbest.shape)

Train feature space before filtering:  (23089, 17068)
Train feature space after filtering:  (23089, 1000)
Test feature space before filtering:  (2753, 17068)
Test feature space after filtering:  (2753, 1000)


In [29]:
all_train_features = cv

indexes = selector.get_support(indices=True)
scores = selector.scores_
feature_names = all_train_features#selector.get_feature_names_out()
print(len(scores), train_vector.shape[1])
print("\n")
print ("Total number of features selected: ", len(indexes))

#Finding informativeness of all the features
informative_features = []
for index in indexes:
    informative_features.append((cv.get_feature_names()[index],scores[index])) 

# Printing all of the features with increment of 40    
print("\n")
print("Informative features: ",informative_features[0::40])

17068 17068


Total number of features selected:  1000


Informative features:  [('abandoned', 45.37718123138039), ('amount', 14.204874243520354), ('bank', 15.127253551444351), ('broken', 172.0753867391924), ('completely', 18.00837380228329), ('deafening', 27.22963818029482), ('discovery', 14.998765682714271), ('employing', 13.613779453228691), ('extend', 24.115893244589213), ('fight', 41.03513713465864), ('gas', 12.340510129853538), ('heat', 11.967255972530156), ('important', 129.66490533951517), ('last', 22.27507278391444), ('mad', 84.81062384671216), ('must', 32.78564993296512), ('overlooked', 12.968822363424248), ('precaution', 12.99404377603876), ('rather', 24.967025147127885), ('said', 54.30047018399957), ('slippery', 14.880930301826986), ('struck', 192.95774606240232), ('thick', 21.469903054379625), ('unimportant', 12.709969241903384), ('weapon', 15.127754677343116)]


In [30]:
# define true labels from train set
x_train = X_train_features_filtered_kbest
y_train = df_train["Label"]
x_test = X_test_features_filtered_kbest
y_test = df_test["Label"]


# fit model and test on train
model = MultinomialNB()
model.fit(x_train, y_train)
predictions = model.predict(x_test)
print ("Accuracy score: ", accuracy_score(y_test, predictions))

Accuracy score:  0.7842353795859063


In [31]:
# For average = WEIGHTED
print ("Precision score: ", precision_score(y_test, predictions, average = 'weighted'))
print ("Recall score: ", recall_score(y_test, predictions, average = 'weighted'))
print ("F1 score: ", f1_score(y_test, predictions, average = 'weighted'))

Precision score:  0.7855095455908955
Recall score:  0.7842353795859063
F1 score:  0.7832239278256331


In [32]:
selector = SelectKBest(chi2, k=5000)
X_train_features_filtered_kbest = selector.fit_transform(train_vector, y_train)
print ("Train feature space before filtering: ", train_vector.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_kbest.shape)


X_test_features_filtered_kbest = selector.transform(test_vector)
print ("Test feature space before filtering: ", test_vector.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_kbest.shape)

Train feature space before filtering:  (23089, 17068)
Train feature space after filtering:  (23089, 5000)
Test feature space before filtering:  (2753, 17068)
Test feature space after filtering:  (2753, 5000)


In [33]:
all_train_features = cv

indexes = selector.get_support(indices=True)
scores = selector.scores_
feature_names = all_train_features#selector.get_feature_names_out()
print(len(scores), train_vector.shape[1])
print("\n")
print ("Total number of features selected: ", len(indexes))

#Finding informativeness of all the features
informative_features = []
for index in indexes:
    informative_features.append((cv.get_feature_names()[index],scores[index])) 

# Printing all of the features with increment of 40    
print("\n")
print("Informative features: ",informative_features[0::40])

17068 17068


Total number of features selected:  5000


Informative features:  [('040', 3.5535778713168975), ('abode', 3.4022957204095228), ('action', 14.797308821260618), ('afterward', 3.5111158971498324), ('alpha', 7.640720872824074), ('animated', 9.17927347559176), ('apron', 3.5535778713168975), ('asia', 3.3594142912589637), ('attractions', 7.877413427739999), ('balls', 5.719511085250525), ('beauties', 8.883944678292243), ('beyond', 3.7367933940553457), ('boldest', 6.354984620951692), ('bread', 8.647207405713722), ('builders', 3.5535778713168975), ('cannibals', 3.4955860591872763), ('cathedral', 5.330366806975345), ('charmed', 8.883944678292243), ('clasped', 4.490753757975559), ('colonists', 14.171324750307086), ('complete', 3.4593725398154023), ('conquer', 3.5742275298532498), ('contradictory', 3.5535778713168975), ('crab', 3.5535778713168975), ('cultivate', 5.330366806975345), ('deadly', 11.647795107622875), ('delicately', 3.5535778713168975), ('despatches', 6.354984620951692), (

In [35]:
# define true labels from train set
x_train = X_train_features_filtered_kbest
y_train = df_train["Label"]
x_test = X_test_features_filtered_kbest
y_test = df_test["Label"]

# fit model and test on trai


model = MultinomialNB()
model.fit(x_train, y_train)
predictions = model.predict(x_test)
print ("Accuracy score: ", accuracy_score(y_test, predictions))

Accuracy score:  0.7490010897203051


In [36]:
# For average = WEIGHTED
print ("Precision score: ", precision_score(y_test, predictions, average = 'weighted'))
print ("Recall score: ", recall_score(y_test, predictions, average = 'weighted'))
print ("F1 score: ", f1_score(y_test, predictions, average = 'weighted'))

Precision score:  0.7517381545104375
Recall score:  0.7490010897203051
F1 score:  0.7477248406748069


#### top k-best features using information gain (mutual information)

In [37]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif

selector = SelectKBest(mutual_info_classif, k='all')
X_train_features_filtered_kbest = selector.fit_transform(train_vector, y_train)
print ("Train feature space before filtering: ", train_vector.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_kbest.shape)


X_test_features_filtered_kbest = selector.transform(test_vector)
print ("Test feature space before filtering: ", test_vector.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_kbest.shape)

Train feature space before filtering:  (23089, 17068)
Train feature space after filtering:  (23089, 17068)
Test feature space before filtering:  (2753, 17068)
Test feature space after filtering:  (2753, 17068)


In [38]:
all_train_features = cv

indexes = selector.get_support(indices=True)
scores = selector.scores_
feature_names = all_train_features#selector.get_feature_names_out()
print(len(scores), train_vector.shape[1])
print("\n")
print ("Total number of features selected: ", len(indexes))


#Finding informativeness of all the features
informative_features = []
for index in indexes:
    informative_features.append((cv.get_feature_names()[index],scores[index])) 

# Printing all of the features with increment of 40    
print("\n")
print("Informative features: ",informative_features[0::400])

17068 17068


Total number of features selected:  17068


Informative features:  [('000', 0.00019664352430067247), ('accidents', 0.00011679184912301034), ('amethysts', 3.963313945708625e-05), ('asks', 6.192472413586096e-05), ('bearded', 4.4234643337808357e-05), ('bounded', 2.570384246491095e-05), ('capes', 2.8548037012295123e-05), ('chop', 4.611615756216982e-05), ('compensate', 8.847262014694901e-05), ('coral', 0.00010235656441370453), ('danes', 3.963313945708625e-05), ('deviation', 6.719316630501284e-05), ('dragonnier', 4.4234643337808357e-05), ('encountered', 6.192472413586096e-05), ('exotic', 4.4234643337808357e-05), ('fight', 0.0007847582696267267), ('francisco', 6.0144146181318934e-05), ('gnawed', 2.7589810939393175e-05), ('hatch', 4.4234643337808357e-05), ('humour', 6.192472413586096e-05), ('inevitable', 4.615110903906036e-05), ('israelites', 0.00013271393097398477), ('laying', 0.0001640115623815666), ('macciota', 4.4234643337808357e-05), ('mercy', 0.003803938505259309), ('mushro

In [39]:
# define true labels from train set
x_train = X_train_features_filtered_kbest
y_train = df_train["Label"]
x_test = X_test_features_filtered_kbest
y_test = df_test["Label"]

# fit model and test on trai


model = MultinomialNB()
model.fit(x_train, y_train)
predictions = model.predict(x_test)
print ("Accuracy score: ", accuracy_score(y_test, predictions))

Accuracy score:  0.7134035597529967


In [40]:
# For average = WEIGHTED
print ("Precision score: ", precision_score(y_test, predictions, average = 'weighted'))
print ("Recall score: ", recall_score(y_test, predictions, average = 'weighted'))
print ("F1 score: ", f1_score(y_test, predictions, average = 'weighted'))

Precision score:  0.7178708065297873
Recall score:  0.7134035597529967
F1 score:  0.711875228116898


In [41]:
selector = SelectKBest(mutual_info_classif, k=1000)
X_train_features_filtered_kbest = selector.fit_transform(train_vector, y_train)
print ("Train feature space before filtering: ", train_vector.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_kbest.shape)


X_test_features_filtered_kbest = selector.transform(test_vector)
print ("Test feature space before filtering: ", test_vector.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_kbest.shape)

Train feature space before filtering:  (23089, 17068)
Train feature space after filtering:  (23089, 1000)
Test feature space before filtering:  (2753, 17068)
Test feature space after filtering:  (2753, 1000)


In [42]:
all_train_features = cv

indexes = selector.get_support(indices=True)
scores = selector.scores_
feature_names = all_train_features#selector.get_feature_names_out()
print(len(scores), train_vector.shape[1])
print("\n")
print ("Total number of features selected: ", len(indexes))


#Finding informativeness of all the features
informative_features = []
for index in indexes:
    informative_features.append((cv.get_feature_names()[index],scores[index])) 

# Printing all of the features with increment of 40    
print("\n")
print("Informative features: ",informative_features[0::40])

17068 17068


Total number of features selected:  1000


Informative features:  [('abandoned', 0.0009075773811769664), ('among', 0.0007621984047898021), ('bank', 0.00040079047440804), ('broke', 0.0012335746446016393), ('columns', 0.0002935691512153299), ('dazzling', 0.0008173905882450157), ('disappeared', 0.003216275887849054), ('enemies', 0.0004480174947779716), ('failure', 0.0002915243038910063), ('fish', 0.0005653979559597092), ('god', 0.0006099522800201148), ('hesitate', 0.0006540345674751857), ('indeed', 0.0003133661403603026), ('leagues', 0.0003463417758651842), ('many', 0.0007850142045683186), ('nearing', 0.0003283717671786302), ('painful', 0.0013743890350023527), ('precious', 0.0017213787321073352), ('rage', 0.0009258646493617498), ('safe', 0.0008173905882450157), ('sigh', 0.00026837318962841304), ('stranger', 0.00026725211314956843), ('tears', 0.00039087966034559863), ('uncle', 0.0002734252957858946), ('wealth', 0.0003539705144401193)]


In [43]:
# define true labels from train set
x_train = X_train_features_filtered_kbest
y_train = df_train["Label"]
x_test = X_test_features_filtered_kbest
y_test = df_test["Label"]

# fit model and test on trai


model = MultinomialNB()
model.fit(x_train, y_train)
predictions = model.predict(x_test)
print ("Accuracy score: ", accuracy_score(y_test, predictions))

Accuracy score:  0.7791500181620051


In [44]:
# For average = WEIGHTED
print ("Precision score: ", precision_score(y_test, predictions, average = 'weighted'))
print ("Recall score: ", recall_score(y_test, predictions, average = 'weighted'))
print ("F1 score: ", f1_score(y_test, predictions, average = 'weighted'))

Precision score:  0.7809343464895393
Recall score:  0.7791500181620051
F1 score:  0.7776538240890853


In [45]:
selector = SelectKBest(mutual_info_classif, k=5000)
X_train_features_filtered_kbest = selector.fit_transform(train_vector, y_train)
print ("Train feature space before filtering: ", train_vector.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_kbest.shape)


X_test_features_filtered_kbest = selector.transform(test_vector)
print ("Test feature space before filtering: ", test_vector.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_kbest.shape)

Train feature space before filtering:  (23089, 17068)
Train feature space after filtering:  (23089, 5000)
Test feature space before filtering:  (2753, 17068)
Test feature space after filtering:  (2753, 5000)


In [46]:
all_train_features = cv

indexes = selector.get_support(indices=True)
scores = selector.scores_
feature_names = all_train_features#selector.get_feature_names_out()
print(len(scores), train_vector.shape[1])
print("\n")
print ("Total number of features selected: ", len(indexes))


#Finding informativeness of all the features
informative_features = []
for index in indexes:
    informative_features.append((cv.get_feature_names()[index],scores[index])) 

# Printing all of the features with increment of 40    
print("\n")
print("Informative features: ",informative_features[0::40])

17068 17068


Total number of features selected:  5000


Informative features:  [('000', 0.00019664352430067247), ('9th', 0.00014211412362858896), ('accustomed', 0.0002526704599753872), ('advice', 0.0002553893566084901), ('allowed', 0.00012371221530067887), ('anger', 0.0009197525593862721), ('applauded', 9.721258547834246e-05), ('arriving', 9.688463579747596e-05), ('atlas', 0.0001104697327908204), ('awkward', 0.0002477346722230848), ('basalt', 8.815652519545336e-05), ('beheld', 9.223785820283606e-05), ('black', 0.00016657636564632072), ('bone', 8.041143119486677e-05), ('bread', 0.0001743607565391738), ('build', 0.00014509667843034288), ('cannibal', 8.847262014694901e-05), ('castle', 9.721258547834246e-05), ('chair', 9.080553448328923e-05), ('cinnamon', 8.847262014694901e-05), ('club', 0.002069510560233155), ('comet', 0.00014472355136925757), ('conceptions', 8.847262014694901e-05), ('consequences', 0.00019582334240472515), ('contrivances', 8.880346647791417e-05), ('countrymen', 0.000176

In [48]:
# define true labels from train set
x_train = X_train_features_filtered_kbest
y_train = df_train["Label"]
x_test = X_test_features_filtered_kbest
y_test = df_test["Label"]

# fit model and test on trai


model = MultinomialNB()
model.fit(x_train, y_train)
predictions = model.predict(x_test)
print ("Accuracy score: ", accuracy_score(y_test, predictions))

Accuracy score:  0.737740646567381


In [49]:
# For average = WEIGHTED
print ("Precision score: ", precision_score(y_test, predictions, average = 'weighted'))
print ("Recall score: ", recall_score(y_test, predictions, average = 'weighted'))
print ("F1 score: ", f1_score(y_test, predictions, average = 'weighted'))

Precision score:  0.7406946436501879
Recall score:  0.737740646567381
F1 score:  0.7364409140404549


In [50]:
# Reading positive words list from text file
pos_words=pd.read_csv('positive_words_list.txt', sep=' ', header=None)
# Converting pandas dataframe to list
pos = pos_words[0].to_list()
# Reading negative words list from text file
neg_words=pd.read_csv('negative_words_list.txt', sep=' ', header=None)
# Converting pandas dataframe to list
neg = neg_words[0].to_list()
# Concatinating two lists and sorting alphabetically
vocab = pos + neg
vocab.sort()
# Removing duplicate words from list
final_vocab = []
for i in vocab:
    if i not in final_vocab:
        final_vocab.append(i)

In [51]:
train_proc_review = df_train['StopwordRemovedSentence']
test_proc_review = df_test['StopwordRemovedSentence']


# Feeding custom vocabulary to the count vectorizer with unigrams
cv = CountVectorizer(ngram_range = [1,1], vocabulary = final_vocab) #for unigrams

In [52]:
# Fitting vectorizer to the train and test dataset

train_vector = cv.fit_transform(train_proc_review)
print("Train Vector Shape:",train_vector.shape)

test_vector = cv.transform(test_proc_review)
print("Test Vector Shape:", test_vector.shape)

Train Vector Shape: (23089, 6786)
Test Vector Shape: (2753, 6786)


In [53]:
selector = SelectKBest(chi2, k='all')
X_train_features_filtered_kbest = selector.fit_transform(train_vector, y_train)
print ("Train feature space before filtering: ", train_vector.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_kbest.shape)


X_test_features_filtered_kbest = selector.transform(test_vector)
print ("Test feature space before filtering: ", test_vector.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_kbest.shape)

Train feature space before filtering:  (23089, 6786)
Train feature space after filtering:  (23089, 6786)
Test feature space before filtering:  (2753, 6786)
Test feature space after filtering:  (2753, 6786)


In [54]:
all_train_features = cv

indexes = selector.get_support(indices=True)
scores = selector.scores_
feature_names = all_train_features#selector.get_feature_names_out()
print(len(scores), train_vector.shape[1])
print("\n")
print ("Total number of features selected: ", len(indexes))


#Finding informativeness of all the features
informative_features = []
for index in indexes:
    informative_features.append((cv.get_feature_names()[index],scores[index])) 

# Printing all of the features with increment of 400    
print("\n")
print("Informative features: ",informative_features[0::400])

6786 6786


Total number of features selected:  6786


Informative features:  [('2-faced', nan), ('ax', 1.3372051149005162), ('caustically', nan), ('damaged', 3.4955860591872763), ('discourage', nan), ('energetic', 14.530263580469146), ('firmer', 1.7767889356584488), ('hallmark', nan), ('impulsive', 3.2921481828504704), ('irresolvable', nan), ('menace', 1.477140623067147), ('overdue', nan), ('pugnacity', nan), ('safe', 35.61492770300717), ('sour', 1.477140623067147), ('tenaciously', nan), ('unrelentingly', nan)]


In [56]:
# define true labels from train set
x_train = X_train_features_filtered_kbest
y_train = df_train["Label"]
x_test = X_test_features_filtered_kbest
y_test = df_test["Label"]

# fit model and test on trai


model = MultinomialNB()
model.fit(x_train, y_train)
predictions = model.predict(x_test)
print ("Accuracy score: ", accuracy_score(y_test, predictions))

Accuracy score:  0.7021431166000727


In [57]:
# For average = WEIGHTED
print ("Precision score: ", precision_score(y_test, predictions, average = 'weighted'))
print ("Recall score: ", recall_score(y_test, predictions, average = 'weighted'))
print ("F1 score: ", f1_score(y_test, predictions, average = 'weighted'))

Precision score:  0.7343152189428906
Recall score:  0.7021431166000727
F1 score:  0.6970377882606712


In [58]:
selector = SelectKBest(chi2, k=1000)
X_train_features_filtered_kbest = selector.fit_transform(train_vector, y_train)
print ("Train feature space before filtering: ", train_vector.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_kbest.shape)


X_test_features_filtered_kbest = selector.transform(test_vector)
print ("Test feature space before filtering: ", test_vector.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_kbest.shape)

Train feature space before filtering:  (23089, 6786)
Train feature space after filtering:  (23089, 1000)
Test feature space before filtering:  (2753, 6786)
Test feature space after filtering:  (2753, 1000)


In [59]:
all_train_features = cv

indexes = selector.get_support(indices=True)
scores = selector.scores_
feature_names = all_train_features#selector.get_feature_names_out()
print(len(scores), train_vector.shape[1])
print("\n")
print ("Total number of features selected: ", len(indexes))


#Finding informativeness of all the features
informative_features = []
for index in indexes:
    informative_features.append((cv.get_feature_names()[index],scores[index])) 

# Printing all of the features with increment of 40    
print("\n")
print("Informative features: ",informative_features[0::40])

6786 6786


Total number of features selected:  1000


Informative features:  [('absence', 3.696569037965878), ('anxiety', 42.380979569926275), ('betray', 8.923333180654398), ('charming', 33.75898977751052), ('cool', 17.39915329121208), ('denied', 14.880930301826986), ('disorder', 25.419938483806767), ('energetic', 14.530263580469146), ('fairly', 5.3771442732179775), ('foolish', 9.50635192921856), ('glory', 21.32146722790138), ('hazard', 6.354984620951692), ('impatient', 38.94370150586465), ('insane', 3.4955860591872763), ('leading', 8.343899245796907), ('masterpiece', 10.66073361395069), ('obstruction', 6.354984620951692), ('powerless', 17.93423901862186), ('recoil', 4.18976536873976), ('rocky', 4.374754120266103), ('sinking', 5.257723908300642), ('strongest', 3.5535778713168975), ('suspicion', 8.923333180654398), ('troubled', 43.71689860320794), ('vomiting', 3.5742275298532498)]


In [61]:
# define true labels from train set
x_train = X_train_features_filtered_kbest
y_train = df_train["Label"]
x_test = X_test_features_filtered_kbest
y_test = df_test["Label"]

# fit model and test on trai


model = MultinomialNB()
model.fit(x_train, y_train)
predictions = model.predict(x_test)
print ("Accuracy score: ", accuracy_score(y_test, predictions))

Accuracy score:  0.683617871413004


In [62]:
# For average = WEIGHTED
print ("Precision score: ", precision_score(y_test, predictions, average = 'weighted'))
print ("Recall score: ", recall_score(y_test, predictions, average = 'weighted'))
print ("F1 score: ", f1_score(y_test, predictions, average = 'weighted'))

Precision score:  0.7493431872016411
Recall score:  0.683617871413004
F1 score:  0.6751972125284392


In [63]:
selector = SelectKBest(chi2, k=5000)
X_train_features_filtered_kbest = selector.fit_transform(train_vector, y_train)
print ("Train feature space before filtering: ", train_vector.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_kbest.shape)


X_test_features_filtered_kbest = selector.transform(test_vector)
print ("Test feature space before filtering: ", test_vector.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_kbest.shape)

Train feature space before filtering:  (23089, 6786)
Train feature space after filtering:  (23089, 5000)
Test feature space before filtering:  (2753, 6786)
Test feature space after filtering:  (2753, 5000)


In [64]:
all_train_features = cv

indexes = selector.get_support(indices=True)
scores = selector.scores_
feature_names = all_train_features#selector.get_feature_names_out()
print(len(scores), train_vector.shape[1])
print("\n")
print ("Total number of features selected: ", len(indexes))


#Finding informativeness of all the features
informative_features = []
for index in indexes:
    informative_features.append((cv.get_feature_names()[index],scores[index])) 

# Printing all of the features with increment of 40    
print("\n")
print("Informative features: ",informative_features[0::40])

6786 6786


Total number of features selected:  5000


Informative features:  [('abnormal', 1.7767889356584488), ('advantageously', 0.6368534274918176), ('amiable', 3.015903072245234), ('approval', 3.5535778713168975), ('awe', 0.8505859162354883), ('bitter', 25.419938483806767), ('brilliance', 5.330366806975345), ('celebrated', 26.651834034876728), ('coldly', 8.178551022852773), ('congested', 6.354984620951692), ('crack', 1.7767889356584488), ('damaged', 3.4955860591872763), ('degenerate', 1.7767889356584488), ('desperate', 28.975017133273553), ('disable', 1.7767889356584488), ('distaste', 3.177492310475846), ('dynamic', 1.5346912206529932), ('enemy', 37.494766911783515), ('exceed', 0.22303411238730753), ('fabrication', 1.5346912206529932), ('fatigued', 11.822370946870219), ('flee', 1.3372051149005162), ('freeze', 1.3372051149005162), ('gasp', 0.15039972181982697), ('graciously', 2.3122625523962825), ('gusto', 5.330366806975345), ('harassment', nan), ('hasty', 3.5742275298532498), ('he

In [65]:
# define true labels from train set
x_train = X_train_features_filtered_kbest
y_train = df_train["Label"]
x_test = X_test_features_filtered_kbest
y_test = df_test["Label"]

# fit model and test on trai


model = MultinomialNB()
model.fit(x_train, y_train)
predictions = model.predict(x_test)
print ("Accuracy score: ", accuracy_score(y_test, predictions))

Accuracy score:  0.7021431166000727


In [66]:
# For average = WEIGHTED
print ("Precision score: ", precision_score(y_test, predictions, average = 'weighted'))
print ("Recall score: ", recall_score(y_test, predictions, average = 'weighted'))
print ("F1 score: ", f1_score(y_test, predictions, average = 'weighted'))

Precision score:  0.7361266046878774
Recall score:  0.7021431166000727
F1 score:  0.6971846847376846
